In [25]:
import pandas as pd

licenses_df = pd.read_excel("../data/licenses.xlsx")
licenses_df.head()
licenses_df.columns

Index(['minutes_date', 'license_number', 'business_name', 'dba_name',
       'address', 'street_number', 'street_name', 'city', 'state', 'zipcode',
       'alcohol_type', 'manager', 'attorney', 'status', 'status_detail',
       'details', 'entity_number', 'file_name'],
      dtype='str')

In [26]:
# Tracker New Bill Licenses
applying_bus=["Crystal Spoons, LLC", "El Centro", "La Parada Dominican Kitchen", "Murl's Kitchen", "JustBook-ish", 
"Minina Cafe", "One Family Diner", "Fresh Food Generation", "Blue Mountain Jamaican Restaurant", "Angelas Cafe", 
"Cabana Grill", "Caffe Dello Sport Eastie", "Con Sabor A Colombia, Inc.", "Florenza", "Mi Pueblito Restaurant", 
"Peruvian Taste Restaurant", "BB.Q Chicken", "Don Tequeno y Dona Arepa", "Mangu Dominican Bistro", "Momo Masala", 
"Stoked Pizza", "Harry's All American", "Mi Finca Mexican Restaurant and Pizzeria", "Las Delicias Colombianas 2", "Little Haiti International Cuisine", "Huntington Theatre", 
"Jeffries Yacht Club", "Zoo New England", "SubRosa and Humaari", "El Barrio Mexican Grill", "Mesob Restaurant", "Mondo", 
"Wash El Beverages, LLC", "District 7 Cafe", "The Mix Vault", "The Weston Way, LLC", "Cool Shade Jamaican Restaurant, LLC", "Blasi Cafe", 
"Milkweed", "Pizza 24", "Doune & Pepe", "Molinaris", "Chilacates Cantina", "Mofongo Factory Restaurant", "El Jardin", 
"Los Alebrijes Restaurant", "Taqueria Jalisco", "The Smoke Shop BBQ", "La Tavernetta by Mida", "AAA Restaurant", "Jadu", 
"Miami Restaurant", "Mr Drinky", "The Green T Coffee Shop", "Knoll Street Tavern", "sweeties", "Somtum Modern Thai Cuisine", 
"Boston Pickle Club", "Capri", "Bono Restaurant and Catering", "American Repertory Theatre", "Boston Center for the Arts", 
"Sorrel and Lime, LLC", "The Substation", "From Scratch", "Mos Tavern & Kitchen", "AMA", 
"Gracenote Coffee Boston", "Merengue Express"]


In [27]:
import pandas as pd

# 1. Standardize the lookup set
applying_bus_set = set(name.strip().lower() for name in applying_bus)

# 2. Check for required columns
cols = licenses_df.columns
if "business_name" in cols and "dba_name" in cols:
    
    # 3. Create boolean masks for both columns
    # We strip and lower both for a fair comparison
    mask_bus = licenses_df["business_name"].str.strip().str.lower().isin(applying_bus_set)
    mask_dba = licenses_df["dba_name"].str.strip().str.lower().isin(applying_bus_set)
    
    # 4. Filter DF where either column matches (| is the OR operator)
    matched_df = licenses_df[mask_bus | mask_dba].copy()
    
    # 5. Calculate missed businesses
    # Collect all names found in either column of our results
    found_bus_names = set(matched_df["business_name"].str.strip().str.lower())
    found_dba_names = set(matched_df["dba_name"].str.strip().str.lower())
    all_found_names = found_bus_names.union(found_dba_names)
    
    missed_names = [name for name in applying_bus if name.strip().lower() not in all_found_names]
    
    # 6. Print Results
    print(f"Found {len(matched_df)} matching rows.")
    print(f"Unique businesses matched: {len(applying_bus) - len(missed_names)} out of {len(applying_bus)}")
    
    if missed_names:
        print(f"\nMissed Businesses ({len(missed_names)}):")
        for name in missed_names:
            print(f" - {name}")
            
    # Display the result
    #display(matched_df) # Use display() in notebooks for better formatting
    
else:
    missing = [c for c in ["business_name", "dba_name"] if c not in licenses_df.columns]
    print(f"Error: Missing columns in licenses_df: {missing}")

Found 98 matching rows.
Unique businesses matched: 69 out of 69


In [31]:
pd.set_option("display.max_rows", None)
#pd.reset_option("display.max_rows")


matched_df['minutes_date'] = pd.to_datetime(matched_df['minutes_date'], errors='coerce')

matched_df = (
    matched_df
    .sort_values(by='minutes_date', ascending=False)
    .drop_duplicates(subset='license_number', keep='first')
)

matched_df = matched_df.sort_values(
    by=["business_name", "dba_name"],
    ascending=[True, True],
    key=lambda col: col.str.lower()
)

cols = ["business_name", "dba_name", "alcohol_type","license_number"]
existing_cols = [c for c in cols if c in matched_df.columns]

matched_df[existing_cols]


,business_name,dba_name,alcohol_type,license_number
278,1010 Morrissey Corp.,Milkweed,all alcoholic beverages,LB-596016
297,1010 Morrissey Corp.,Milkweed,all alcoholic beverages,LB-595701
279,"490 Harrison Restaurant, LLC",Capri,all alcoholic beverages,LB-595024
567,"567 Washington St, LLC",SubRosa and Humaari,all alcoholic beverages,LB-577208
351,"AAA Restaurant, LLC",AAA Restaurant,all alcoholic beverages,LB-592817
1261,"Allston BBQ, LLC",BB.Q Chicken,all alcoholic beverages,LB-99639
460,Amabaka Corporation,Momo Masala,all alcoholic beverages,LB-582324
231,"American Repertory Theatre Company, Inc.",American Repertory Theatre,all alcoholic beverages,LB-599561
542,"Angelas Cafe II, Inc.",Angelas Cafe,all alcoholic beverages,LB-578145
404,"ATCF, LLC",Knoll Street Tavern,all alcoholic beverages,LB-587892


In [8]:
# Tracking Upgrades
import pandas as pd

# 1. Your input list
UPGRADES = ["LB-99619", "LB-99779", "LB-99607", "LB-99709", "LB-99753"]

# 2. Filter the DataFrame
# We use .copy() to avoid SettingWithCopyWarnings later
matched_df = licenses_df[licenses_df['license_number'].isin(UPGRADES)].copy()

# 3. Data Cleaning: Convert 'minutes_date' to actual datetime objects
# errors='coerce' will turn unparseable dates into NaT (Not a Time) instead of crashing
matched_df['minutes_date'] = pd.to_datetime(matched_df['minutes_date'], errors='coerce')

# 4. Sort and Deduplicate
# - Sort by date descending (Newest at the top)
# - Drop duplicates of the license_number, keeping only the first (the newest)
final_upgrades_df = (
    matched_df
    .sort_values(by='minutes_date', ascending=False)
    .drop_duplicates(subset='license_number', keep='first')
)

# 5. Calculate Metrics for Reporting
found_count = len(final_upgrades_df)
total_requested = len(UPGRADES)
found_ids = set(final_upgrades_df['license_number'].unique())
missed_ids = [uid for uid in UPGRADES if uid not in found_ids]

# 6. Print Summary Report
print("-" * 30)
print(f"UPGRADE MATCHING REPORT")
print("-" * 30)
print(f"Total IDs requested: {total_requested}")
print(f"Unique matches found: {found_count}")

if missed_ids:
    print(f"\n❌ Missed IDs ({len(missed_ids)}):")
    for mid in missed_ids:
        print(f" - {mid}")
else:
    print("\n✅ All IDs successfully matched!")

print("-" * 30)

# 7. Display the result (newest first)
# Selecting specific columns for a cleaner view in the notebook
view_cols = ['license_number', 'minutes_date', 'business_name', 'dba_name']
# Only show columns that actually exist in your dataframe
existing_cols = [c for c in view_cols if c in final_upgrades_df.columns]

final_upgrades_df[existing_cols].sort_values(by='minutes_date', ascending=False)

------------------------------
UPGRADE MATCHING REPORT
------------------------------
Total IDs requested: 5
Unique matches found: 5

✅ All IDs successfully matched!
------------------------------


,license_number,minutes_date,business_name,dba_name
2,LB-99607,2026-01-28,"Bop Productions, LLC",Bebop
10,LB-99779,2026-01-28,"Nite Lite Cafe, LLC",89 Charles
13,LB-99753,2026-01-28,"Schiaffo, Inc.",Carmelinas
15,LB-99619,2026-01-28,"SMG Newbury Restaurant, LLC",Serafina
18,LB-99709,2026-01-28,"Wichcraft, LLC",New England Wicked Craft Company
